<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [5]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from two_opt import Two_opt
from pyrat import BigHolesRandomMaze, Action,MazeFromDict

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [6]:
class Two_optTests (unittest.TestCase):

    """
        This class tests the methods of the Greedy class.
        For each method, we test it with a few different configurations.
    """

    #############################################################################################################################################
    #                                                                 UNIT TESTS                                                                #
    #############################################################################################################################################

    def test_traversal ( self: Self
                       ) ->    None:

        """
            This method tests the 'traversal' method.
            We are going to check the following:
            - Outputs are of the expected types.
            - All vertices are visited.
            - The routing table is a tree, with root corresponding to the start of the traversal.
            - The found distances are correct, i.e., strictly positive and increasing, except for the start vertex which should be 0.
            Note that we cannot test that the output is unique, as there are multiple valid outputs, depending on how vertices are explored.
            We will test the method on several random graphs to make sure it is working properly.
            Random graphs will be generated using the PyRat class used to create mazes in games.
            There are several such classes, but we will use the BigHolesRandomMaze class.
            In:
                * self: Reference to the current object.
            Out:
                * None.
        """

        # Constants
        NB_GRAPHS = 10
        WIDTHS = [2, 30]
        HEIGHTS = [2, 30]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        # Test on several graphs
        for i in range(NB_GRAPHS):
            
            # Instantiate the player
            player = Two_opt()

            # Generate a random maze
            # We use a fixed random seed for reproducibility of tests
            rng = random.Random(i)
            maze = BigHolesRandomMaze(width = rng.randint(WIDTHS[0], WIDTHS[1]),
                                      height = rng.randint(HEIGHTS[0], HEIGHTS[1]),
                                      cell_percentage = rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                                      wall_percentage = rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                                      mud_percentage = MUD_PERCENTAGE,
                                      random_seed = i)

            # Choose a random starting vertex
            start_vertex = rng.choice(maze.vertices)

            # Perform the traversal
            distances, routing_table = player.traversal(maze, start_vertex)

            # Check the output type for distances
            # It should be a dictionary with integer keys and integer values
            self.assertTrue(isinstance(distances, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in distances.keys()))
            self.assertTrue(all(isinstance(v, Integral) for v in distances.values()))
            
            # Check the output type for the routing table
            # It should be a dictionary with integer keys and integer or None values
            self.assertTrue(isinstance(routing_table, Dict))
            self.assertTrue(all(isinstance(k, Integral) for k in routing_table.keys()))
            self.assertTrue(all(isinstance(v, (type(None), Integral)) for v in routing_table.values()))

            # All vertices should be visited
            # Equivalently, the distances should have the same keys as the maze vertices
            self.assertEqual(sorted(set(distances.keys())), sorted(maze.vertices))

            # Check that the start vertex is the only tree root
            self.assertEqual(routing_table[start_vertex], None)
            self.assertTrue(all(routing_table[v] is not None for v in routing_table.keys() if v != start_vertex))
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] > 0 for v in distances.keys() if v != start_vertex))
            
            # We check that the routing table is a tree
            # Also we check that associated distances are decreasing as we go to the root
            for vertex in routing_table:

                # We check that the parent is closer to the root
                if routing_table[vertex] is not None:
                    self.assertTrue(distances[routing_table[vertex]] < distances[vertex])
                
                # We ckeck that we can reach the root from any vertex
                path = [vertex]
                while routing_table[path[-1]] is not None:
                    self.assertTrue(routing_table[path[-1]] not in path)
                    path.append(routing_table[path[-1]])

    def test_traversal_fixed ( self: Self
                             ) ->    None:

            """
                This method complements the previous tests by testing the 'traversal' method on some controlled examples.
                To do so, we will use some mazes for which there is a unique routing table.
                We do not test the output types again, as it has already been done in the previous test function.
                In:
                    * self: Reference to the current object.
                Out:
                    * None.
            """

            # Constants
            TESTS = [{"inputs": {"maze": MazeFromDict({0: {1: 1, 20: 1}, 1: {0: 1, 2: 1, 21: 1}, 2: {1: 1, 3: 1, 22: 1}, 3: {2: 1, 4: 1, 23: 1}, 4: {3: 1, 5: 1, 24: 1}, 5: {4: 1, 6: 1, 25: 1}, 6: {5: 1, 26: 1, 7: 1}, 7: {8: 1, 27: 1, 6: 1}, 8: {7: 1, 9: 1}, 9: {8: 1, 10: 1}, 10: {9: 1, 11: 1, 30: 1}, 11: {10: 1, 12: 1, 31: 1}, 12: {11: 1, 13: 1, 32: 1}, 13: {12: 1, 14: 1, 33: 1}, 14: {13: 1, 15: 1}, 15: {14: 1, 16: 1, 35: 8}, 16: {15: 1, 17: 1, 36: 1}, 17: {16: 1, 18: 4}, 18: {19: 1, 38: 1, 17: 4}, 19: {18: 1, 39: 1}, 20: {0: 1, 21: 1, 40: 6}, 21: {1: 1, 41: 1, 22: 1, 20: 1}, 22: {2: 1, 21: 1, 23: 1}, 23: {3: 1, 22: 1, 43: 1}, 24: {4: 1, 44: 1, 25: 4}, 25: {5: 1, 45: 1, 26: 1, 24: 4}, 26: {6: 1, 27: 1, 46: 1, 25: 1}, 27: {7: 1, 26: 1, 28: 1, 47: 6}, 28: {27: 1, 29: 1, 48: 1}, 29: {28: 1, 49: 1, 30: 1}, 30: {10: 1, 50: 1, 29: 1, 31: 7}, 31: {11: 1, 32: 1, 51: 1, 30: 7}, 32: {31: 1, 33: 1, 12: 1, 52: 4}, 33: {53: 1, 13: 1, 32: 1}, 34: {54: 1}, 35: {55: 1, 36: 1, 15: 8}, 36: {37: 1, 16: 1, 35: 1, 56: 1}, 37: {36: 1, 57: 1, 38: 1}, 38: {58: 1, 37: 1, 18: 1, 39: 6}, 39: {19: 1, 59: 1, 38: 6}, 40: {41: 1, 60: 1, 20: 6}, 41: {21: 1, 40: 1, 61: 1}, 42: {43: 1, 62: 1}, 43: {42: 1, 44: 1, 63: 1, 23: 1}, 44: {43: 1, 45: 1, 24: 1}, 45: {44: 1, 46: 1, 25: 1}, 46: {26: 1, 45: 1, 47: 1, 66: 1}, 47: {46: 1, 48: 4, 27: 6}, 48: {68: 1, 28: 1, 49: 1, 47: 4}, 49: {69: 1, 29: 1, 48: 1, 50: 1}, 50: {30: 1, 70: 1, 51: 1, 49: 1}, 51: {71: 1, 31: 1, 50: 1, 52: 1}, 52: {53: 1, 51: 1, 32: 4, 72: 9}, 53: {33: 1, 73: 1, 52: 1, 54: 1}, 54: {34: 1, 74: 1, 53: 1}, 55: {35: 1, 56: 1, 75: 5}, 56: {55: 1, 57: 1, 36: 1, 76: 6}, 57: {37: 1, 58: 1, 77: 1, 56: 1}, 58: {38: 1, 57: 1, 59: 1, 78: 1}, 59: {58: 1, 39: 1, 79: 1}, 60: {61: 1, 40: 1, 80: 1}, 61: {60: 1, 62: 1, 81: 1, 41: 1}, 62: {61: 1, 63: 1, 82: 1, 42: 1}, 63: {62: 1, 64: 1, 83: 1, 43: 1}, 64: {63: 1, 65: 1, 84: 1}, 65: {64: 1, 66: 1}, 66: {46: 1, 65: 1, 67: 1, 86: 1}, 67: {66: 1, 87: 1, 68: 1}, 68: {48: 1, 69: 1, 88: 1, 67: 1}, 69: {49: 1, 68: 1, 70: 1, 89: 1}, 70: {69: 1, 50: 1, 71: 1, 90: 1}, 71: {51: 1, 72: 1, 91: 1, 70: 1}, 72: {71: 1, 73: 1, 92: 1, 52: 9}, 73: {53: 1, 72: 1, 74: 1, 93: 1}, 74: {54: 1, 73: 1, 94: 1, 75: 4}, 75: {76: 1, 95: 1, 74: 4, 55: 5}, 76: {75: 1, 77: 1, 96: 1, 56: 6}, 77: {57: 1, 76: 1, 78: 1, 97: 8}, 78: {77: 1, 79: 1, 58: 1, 98: 1}, 79: {78: 1, 99: 1, 59: 1}, 80: {81: 1, 100: 1, 60: 1}, 81: {80: 1, 82: 1, 101: 1, 61: 1}, 82: {81: 1, 83: 1, 102: 1, 62: 1}, 83: {82: 1, 103: 1, 63: 1, 84: 1}, 84: {85: 1, 64: 1, 83: 1, 104: 4}, 85: {84: 1, 86: 1, 105: 1}, 86: {66: 1, 85: 1, 87: 1}, 87: {67: 1, 86: 1, 107: 1, 88: 4}, 88: {89: 1, 108: 1, 68: 1, 87: 4}, 89: {69: 1, 88: 1, 90: 1, 109: 1}, 90: {89: 1, 91: 1, 110: 1, 70: 1}, 91: {71: 1, 90: 1, 92: 1, 111: 7}, 92: {91: 1, 72: 1, 93: 8}, 93: {73: 1, 94: 1, 92: 8, 113: 7}, 94: {93: 1, 114: 1, 74: 1, 95: 1}, 95: {75: 1, 115: 1, 96: 1, 94: 1}, 96: {76: 1, 95: 1, 116: 1, 97: 6}, 97: {117: 1, 98: 7, 96: 6, 77: 8}, 98: {118: 1, 78: 1, 99: 1, 97: 7}, 99: {79: 1, 119: 1, 98: 1}, 100: {80: 1, 120: 1, 101: 1}, 101: {102: 1, 81: 1, 121: 1, 100: 1}, 102: {82: 1, 101: 1, 122: 1}, 103: {83: 1, 123: 1}, 104: {124: 1, 84: 4}, 105: {85: 1, 106: 1, 125: 1}, 106: {105: 1, 126: 1, 107: 1}, 107: {108: 1, 127: 1, 106: 1, 87: 1}, 108: {88: 1, 107: 1, 109: 1}, 109: {89: 1, 129: 1, 108: 1}, 110: {111: 1, 130: 1, 90: 1}, 111: {110: 1, 131: 1, 112: 8, 91: 7}, 112: {132: 1, 113: 1, 111: 8}, 113: {114: 1, 112: 1, 133: 1, 93: 7}, 114: {94: 1, 113: 1, 115: 1, 134: 5}, 115: {95: 1, 116: 1, 135: 1, 114: 1}, 116: {115: 1, 117: 1, 136: 1, 96: 1}, 117: {116: 1, 118: 1, 97: 1, 137: 1}, 118: {117: 1, 138: 1, 98: 1, 119: 1}, 119: {99: 1, 139: 1, 118: 1}, 120: {100: 1, 140: 1, 121: 1}, 121: {122: 1, 141: 1, 101: 1, 120: 1}, 122: {121: 1, 123: 1, 102: 1}, 123: {103: 1, 122: 1, 124: 1, 143: 1}, 124: {104: 1, 123: 1, 144: 1, 125: 1}, 125: {105: 1, 145: 1, 124: 1, 126: 4}, 126: {106: 1, 127: 1, 146: 1, 125: 4}, 127: {107: 1, 128: 1, 126: 1, 147: 1}, 128: {127: 1, 148: 1, 129: 1}, 129: {109: 1, 128: 1, 149: 1}, 130: {150: 1, 110: 1}, 131: {111: 1, 132: 1, 151: 1}, 132: {131: 1, 152: 1, 112: 1, 133: 9}, 133: {153: 1, 113: 1, 134: 1, 132: 9}, 134: {135: 1, 154: 1, 133: 1, 114: 5}, 135: {115: 1, 134: 1, 155: 1, 136: 1}, 136: {116: 1, 137: 1, 135: 1, 156: 5}, 137: {136: 1, 157: 1, 117: 1}, 138: {118: 1, 158: 1, 139: 5}, 139: {159: 1, 119: 1, 138: 5}, 140: {141: 1, 120: 1, 160: 1}, 141: {121: 1, 140: 1, 142: 1, 161: 7}, 142: {143: 1, 141: 1}, 143: {123: 1, 142: 1, 163: 1, 144: 1}, 144: {124: 1, 164: 1, 143: 1}, 145: {125: 1, 146: 1, 165: 1}, 146: {145: 1, 166: 1, 126: 1}, 147: {148: 1, 127: 1, 167: 6}, 148: {128: 1, 147: 1, 168: 1, 149: 1}, 149: {169: 1, 150: 1, 129: 1, 148: 1}, 150: {130: 1, 151: 1, 170: 1, 149: 1}, 151: {131: 1, 150: 1, 152: 1, 171: 8}, 152: {132: 1, 172: 1, 151: 1}, 153: {173: 1, 133: 1}, 154: {174: 1, 134: 1, 155: 8}, 155: {135: 1, 156: 1, 175: 1, 154: 8}, 156: {155: 1, 176: 1, 157: 1, 136: 5}, 157: {137: 1, 158: 1, 156: 1, 177: 1}, 158: {157: 1, 178: 1, 138: 1, 159: 5}, 159: {139: 1, 158: 5}, 160: {161: 1, 180: 1, 140: 1}, 161: {160: 1, 162: 1, 181: 1, 141: 7}, 162: {161: 1, 163: 1, 182: 1}, 163: {162: 1, 164: 1, 143: 1}, 164: {144: 1, 163: 1, 184: 1, 165: 1}, 165: {166: 1, 185: 1, 145: 1, 164: 1}, 166: {146: 1, 165: 1, 186: 1, 167: 1}, 167: {168: 1, 187: 1, 166: 1, 147: 6}, 168: {148: 1, 167: 1, 188: 1, 169: 1}, 169: {149: 1, 170: 1, 189: 1, 168: 1}, 170: {150: 1, 169: 1, 171: 1, 190: 1}, 171: {170: 1, 172: 1, 151: 8}, 172: {171: 1, 173: 1, 192: 1, 152: 1}, 173: {153: 1, 172: 1, 174: 1, 193: 1}, 174: {154: 1, 173: 1, 175: 1, 194: 1}, 175: {174: 1, 155: 1, 195: 1}, 176: {156: 1, 177: 1, 196: 1}, 177: {176: 1, 197: 1, 157: 1, 178: 6}, 178: {158: 1, 179: 1, 198: 7, 177: 6}, 179: {178: 1, 199: 4}, 180: {160: 1, 200: 1, 181: 1}, 181: {182: 1, 201: 1, 161: 1, 180: 1}, 182: {162: 1, 181: 1, 183: 1, 202: 1}, 183: {184: 1, 203: 1, 182: 1}, 184: {164: 1, 183: 1, 204: 1}, 185: {165: 1, 205: 1}, 186: {166: 1, 187: 1, 206: 1}, 187: {186: 1, 188: 1, 167: 1, 207: 1}, 188: {168: 1, 189: 1, 208: 1, 187: 1}, 189: {188: 1, 209: 1, 169: 1}, 190: {170: 1, 191: 1, 210: 1}, 191: {190: 1, 211: 1}, 192: {172: 1, 212: 1, 193: 1}, 193: {173: 1, 194: 1, 213: 1, 192: 1}, 194: {174: 1, 214: 1, 193: 1, 195: 1}, 195: {215: 1, 196: 1, 194: 1, 175: 1}, 196: {195: 1, 176: 1, 216: 5}, 197: {217: 1, 177: 1, 198: 5}, 198: {218: 1, 199: 1, 197: 5, 178: 7}, 199: {219: 1, 198: 1, 179: 4}, 200: {180: 1, 220: 1}, 201: {181: 1, 221: 1}, 202: {203: 1, 182: 1, 222: 5}, 203: {183: 1, 202: 1}, 204: {184: 1, 224: 1, 205: 1}, 205: {206: 1, 225: 1, 185: 1, 204: 1}, 206: {186: 1, 205: 1, 207: 1, 226: 1}, 207: {206: 1, 227: 1, 208: 1, 187: 1}, 208: {209: 1, 228: 1, 188: 1, 207: 1}, 209: {189: 1, 208: 1, 229: 1, 210: 1}, 210: {190: 1, 230: 1, 209: 1, 211: 1}, 211: {191: 1, 210: 1, 212: 1, 231: 8}, 212: {192: 1, 211: 1, 213: 1}, 213: {214: 1, 233: 1, 212: 1, 193: 1}, 214: {194: 1, 213: 1, 215: 1}, 215: {195: 1, 214: 1, 216: 1, 235: 1}, 216: {215: 1, 217: 1, 236: 1, 196: 5}, 217: {197: 1, 216: 1, 237: 1, 218: 1}, 218: {198: 1, 238: 1, 219: 1, 217: 1}, 219: {199: 1, 218: 1, 239: 1}, 220: {221: 1, 240: 1, 200: 1}, 221: {220: 1, 222: 1, 241: 1, 201: 1}, 222: {221: 1, 223: 1, 242: 1, 202: 5}, 223: {222: 1, 224: 1, 243: 1}, 224: {204: 1, 223: 1, 225: 1, 244: 1}, 225: {224: 1, 205: 1, 245: 1}, 226: {206: 1, 246: 1, 227: 1}, 227: {228: 1, 226: 1, 207: 1, 247: 1}, 228: {208: 1, 227: 1, 229: 1}, 229: {209: 1, 249: 1, 230: 1, 228: 1}, 230: {210: 1, 250: 1, 229: 1}, 231: {232: 1, 251: 1, 211: 8}, 232: {231: 1, 233: 1, 252: 1}, 233: {213: 1, 234: 1, 232: 1}, 234: {233: 1, 254: 1, 235: 1}, 235: {236: 1, 255: 1, 215: 1, 234: 1}, 236: {235: 1, 237: 1, 256: 1, 216: 1}, 237: {217: 1, 236: 1, 257: 1, 238: 1}, 238: {258: 1, 237: 1, 218: 1}, 239: {259: 1, 219: 1}, 240: {220: 1, 241: 1}, 241: {242: 1, 240: 1, 221: 1}, 242: {222: 1, 241: 1, 262: 1, 243: 1}, 243: {244: 1, 263: 1, 223: 1, 242: 1}, 244: {224: 1, 243: 1, 245: 1, 264: 1}, 245: {244: 1, 265: 1, 225: 1}, 246: {226: 1, 247: 1, 266: 1}, 247: {246: 1, 248: 1, 267: 1, 227: 1}, 248: {247: 1, 249: 1, 268: 1}, 249: {229: 1, 250: 1, 269: 1, 248: 1}, 250: {249: 1, 270: 1, 230: 1, 251: 8}, 251: {231: 1, 252: 1, 271: 1, 250: 8}, 252: {251: 1, 272: 1, 253: 1, 232: 1}, 253: {254: 1, 273: 1, 252: 1}, 254: {253: 1, 255: 1, 234: 1, 274: 1}, 255: {235: 1, 254: 1, 275: 1, 256: 1}, 256: {257: 1, 276: 1, 255: 1, 236: 1}, 257: {237: 1, 256: 1, 258: 1, 277: 1}, 258: {238: 1, 257: 1, 259: 9}, 259: {239: 1, 279: 1, 258: 9}, 260: {280: 1, 261: 1}, 261: {262: 1, 281: 1, 260: 1}, 262: {242: 1, 261: 1, 282: 1, 263: 1}, 263: {243: 1, 283: 1, 264: 1, 262: 1}, 264: {244: 1, 284: 1, 263: 1}, 265: {266: 1, 245: 1, 285: 4}, 266: {246: 1, 265: 1, 286: 1, 267: 1}, 267: {247: 1, 266: 1, 287: 8}, 268: {288: 1, 269: 1, 248: 1}, 269: {270: 1, 268: 1, 289: 1, 249: 1}, 270: {250: 1, 269: 1, 271: 1, 290: 1}, 271: {251: 1, 270: 1, 291: 1}, 272: {252: 1, 292: 1, 273: 4}, 273: {253: 1, 293: 1, 274: 1, 272: 4}, 274: {275: 1, 294: 1, 273: 1, 254: 1}, 275: {274: 1, 295: 1, 255: 1, 276: 7}, 276: {256: 1, 296: 1, 275: 7}, 277: {257: 1, 278: 1, 297: 1}, 278: {277: 1, 298: 1, 279: 1}, 279: {259: 1, 299: 1, 278: 1}, 280: {260: 1, 281: 1, 300: 1}, 281: {261: 1, 280: 1, 301: 1}, 282: {283: 1, 262: 1, 302: 9}, 283: {263: 1, 282: 1, 303: 1, 284: 1}, 284: {264: 1, 285: 1, 304: 1, 283: 1}, 285: {284: 1, 305: 1, 265: 4}, 286: {266: 1, 287: 1, 306: 6}, 287: {286: 1, 267: 8, 288: 8}, 288: {268: 1, 308: 9, 289: 7, 287: 8}, 289: {269: 1, 309: 1, 290: 1, 288: 7}, 290: {291: 1, 310: 1, 270: 1, 289: 1}, 291: {290: 1, 292: 1, 311: 1, 271: 1}, 292: {272: 1, 291: 1, 312: 1, 293: 9}, 293: {273: 1, 313: 1, 294: 1, 292: 9}, 294: {274: 1, 314: 1, 293: 1, 295: 1}, 295: {296: 1, 315: 1, 275: 1, 294: 1}, 296: {276: 1, 295: 1, 316: 1, 297: 1}, 297: {277: 1, 296: 1, 298: 1, 317: 6}, 298: {278: 1, 318: 1, 297: 1, 299: 1}, 299: {279: 1, 319: 1, 298: 1}, 300: {280: 1, 320: 1, 301: 1}, 301: {281: 1, 302: 1, 300: 1, 321: 5}, 302: {303: 1, 301: 1, 322: 8, 282: 9}, 303: {283: 1, 302: 1, 323: 1}, 304: {284: 1, 324: 1, 305: 1}, 305: {285: 1, 325: 1, 306: 1, 304: 1}, 306: {307: 1, 305: 1, 286: 6, 326: 8}, 307: {308: 1, 306: 1, 327: 1}, 308: {307: 1, 309: 1, 328: 1, 288: 9}, 309: {308: 1, 310: 1, 289: 1, 329: 1}, 310: {309: 1, 311: 1, 330: 1, 290: 1}, 311: {310: 1, 312: 1, 291: 1, 331: 1}, 312: {311: 1, 292: 1, 313: 1, 332: 1}, 313: {293: 1, 314: 1, 312: 1, 333: 7}, 314: {315: 1, 334: 1, 313: 1, 294: 1}, 315: {314: 1, 335: 1, 295: 1, 316: 9}, 316: {296: 1, 336: 1, 315: 9}, 317: {337: 1, 318: 1, 297: 6}, 318: {298: 1, 317: 1, 338: 1, 319: 1}, 319: {299: 1, 318: 1, 339: 1}, 320: {300: 1, 340: 1, 321: 1}, 321: {322: 1, 341: 1, 320: 1, 301: 5}, 322: {321: 1, 342: 1, 323: 1, 302: 8}, 323: {324: 1, 343: 1, 303: 1, 322: 1}, 324: {304: 1, 323: 1, 344: 1, 325: 8}, 325: {326: 1, 305: 1, 345: 1, 324: 8}, 326: {325: 1, 327: 1, 306: 8}, 327: {326: 1, 307: 1, 347: 1, 328: 1}, 328: {308: 1, 329: 1, 348: 1, 327: 1}, 329: {328: 1, 330: 1, 349: 1, 309: 1}, 330: {310: 1, 331: 1, 329: 1, 350: 1}, 331: {351: 1, 330: 1, 311: 1, 332: 1}, 332: {352: 1, 331: 1, 333: 1, 312: 1}, 333: {353: 1, 332: 1, 313: 7}, 334: {314: 1, 335: 1, 354: 1}, 335: {315: 1, 336: 1, 334: 1}, 336: {316: 1, 337: 1, 335: 1, 356: 9}, 337: {317: 1, 336: 1, 338: 6}, 338: {318: 1, 358: 1, 337: 6, 339: 9}, 339: {359: 1, 319: 1, 338: 9}, 340: {320: 1, 360: 1, 341: 1}, 341: {321: 1, 361: 1, 340: 1, 342: 1}, 342: {362: 1, 343: 1, 322: 1, 341: 1}, 343: {344: 1, 363: 1, 342: 1, 323: 1}, 344: {324: 1, 343: 1, 345: 1, 364: 1}, 345: {344: 1, 346: 1, 365: 1, 325: 1}, 346: {345: 1, 366: 1}, 347: {367: 1, 327: 1, 348: 5}, 348: {328: 1, 349: 1, 368: 1, 347: 5}, 349: {348: 1, 329: 1, 350: 1, 369: 1}, 350: {349: 1, 330: 1, 351: 9}, 351: {331: 1, 352: 1, 371: 1, 350: 9}, 352: {332: 1, 351: 1, 353: 9, 372: 5}, 353: {333: 1, 354: 1, 373: 1, 352: 9}, 354: {353: 1, 355: 1, 374: 1, 334: 1}, 355: {354: 1, 356: 1}, 356: {355: 1, 357: 1, 376: 1, 336: 9}, 357: {356: 1, 358: 1, 377: 1}, 358: {357: 1, 359: 1, 338: 1, 378: 1}, 359: {339: 1, 358: 1, 379: 1}, 360: {361: 1, 380: 1, 340: 1}, 361: {341: 1, 360: 1, 381: 1, 362: 1}, 362: {342: 1, 363: 1, 382: 1, 361: 1}, 363: {362: 1, 364: 1, 383: 1, 343: 1}, 364: {344: 1, 363: 1, 384: 1, 365: 1}, 365: {345: 1, 366: 1, 385: 1, 364: 1}, 366: {367: 1, 365: 1, 346: 1, 386: 1}, 367: {347: 1, 366: 1, 387: 1, 368: 1}, 368: {348: 1, 369: 1, 388: 1, 367: 1}, 369: {368: 1, 389: 1, 370: 1, 349: 1}, 370: {371: 1, 390: 1, 369: 1}, 371: {351: 1, 370: 1, 372: 1, 391: 1}, 372: {373: 1, 392: 1, 371: 1, 352: 5}, 373: {372: 1, 374: 1, 393: 1, 353: 1}, 374: {354: 1, 373: 1, 375: 1, 394: 5}, 375: {376: 1, 374: 1, 395: 1}, 376: {356: 1, 375: 1, 377: 1, 396: 1}, 377: {376: 1, 397: 1, 357: 1, 378: 7}, 378: {379: 1, 398: 1, 358: 1, 377: 7}, 379: {378: 1, 399: 1, 359: 1}, 380: {360: 1}, 381: {382: 1, 361: 1}, 382: {362: 1, 381: 1, 383: 1}, 383: {382: 1, 363: 1}, 384: {364: 1, 385: 1}, 385: {384: 1, 386: 1, 365: 1}, 386: {385: 1, 387: 1, 366: 1}, 387: {367: 1, 386: 1, 388: 7}, 388: {368: 1, 389: 1, 387: 7}, 389: {369: 1, 390: 1, 388: 1}, 390: {370: 1, 391: 1, 389: 1}, 391: {390: 1, 392: 1, 371: 1}, 392: {372: 1, 393: 1, 391: 1}, 393: {373: 1, 392: 1, 394: 1}, 394: {393: 1, 395: 1, 374: 5}, 395: {396: 1, 394: 1, 375: 1}, 396: {376: 1, 395: 1, 397: 1}, 397: {377: 1, 398: 1, 396: 1}, 398: {378: 1, 397: 1, 399: 1}, 399: {379: 1, 398: 1}}),
                                "start": 18,
                                "peices_of_cheese":[342,145,165,122,132,127,190]},
                    "outputs": {"meta_graph":{(18, 342): ([18, 38, 37, 57, 77, 76, 75, 95, 94, 114, 113, 112, 132, 131, 151, 150, 149, 148, 168, 167, 166, 165, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 33), (18, 145): ([18, 38, 37, 36, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 26, 46, 66, 86, 85, 105, 125, 145], 22), (18, 165): ([18, 38, 37, 57, 77, 76, 75, 95, 94, 114, 113, 112, 132, 131, 151, 150, 149, 148, 168, 167, 166, 165], 21), (18, 122): ([18, 38, 37, 36, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 23, 43, 42, 62, 82, 102, 122], 24), (18, 132): ([18, 38, 37, 57, 77, 76, 75, 95, 94, 114, 113, 112, 132], 12), (18, 127): ([18, 38, 37, 36, 16, 15, 14, 13, 12, 11, 10, 30, 29, 28, 48, 68, 67, 87, 107, 127], 19), (18, 190): ([18, 38, 37, 57, 77, 76, 75, 95, 94, 114, 113, 112, 132, 131, 151, 150, 170, 190], 17), (190, 342): ([190, 210, 209, 208, 207, 206, 205, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 16), (190, 145): ([190, 170, 169, 168, 167, 166, 146, 145], 7), (190, 165): ([190, 170, 169, 168, 167, 166, 165], 6), (190, 122): ([190, 170, 169, 168, 167, 166, 146, 145, 125, 124, 123, 122], 11), (190, 132): ([190, 170, 150, 151, 131, 132], 5), (190, 127): ([190, 170, 150, 149, 129, 128, 127], 6), (127, 342): ([127, 126, 146, 145, 165, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 16), (127, 145): ([127, 126, 146, 145], 3), (127, 165): ([127, 126, 146, 145, 165], 4), (127, 122): ([127, 107, 106, 105, 125, 124, 123, 122], 7), (127, 132): ([127, 128, 129, 149, 150, 151, 131, 132], 7), (132, 342): ([132, 131, 151, 150, 149, 148, 168, 167, 166, 165, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 21), (132, 145): ([132, 131, 151, 150, 149, 129, 128, 127, 126, 146, 145], 10), (132, 165): ([132, 131, 151, 150, 149, 148, 168, 167, 166, 165], 9), (132, 122): ([132, 131, 111, 110, 90, 89, 88, 108, 107, 106, 105, 125, 124, 123, 122], 14), (122, 342): ([122, 123, 124, 144, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 15), (122, 145): ([122, 123, 124, 125, 145], 4), (122, 165): ([122, 123, 124, 125, 145, 165], 5), (165, 342): ([165, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 12), (165, 145): ([165, 145], 1), (145, 342): ([145, 165, 164, 184, 204, 224, 223, 243, 263, 283, 303, 323, 322, 342], 13), (342, 18): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 165, 166, 167, 168, 148, 149, 150, 151, 131, 132, 112, 113, 114, 94, 95, 75, 76, 77, 57, 37, 38, 18], 33), (145, 18): ([145, 125, 105, 85, 86, 66, 46, 26, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 36, 37, 38, 18], 22), (165, 18): ([165, 166, 167, 168, 148, 149, 150, 151, 131, 132, 112, 113, 114, 94, 95, 75, 76, 77, 57, 37, 38, 18], 21), (122, 18): ([122, 102, 82, 62, 42, 43, 23, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 36, 37, 38, 18], 24), (132, 18): ([132, 112, 113, 114, 94, 95, 75, 76, 77, 57, 37, 38, 18], 12), (127, 18): ([127, 107, 87, 67, 68, 48, 28, 29, 30, 10, 11, 12, 13, 14, 15, 16, 36, 37, 38, 18], 19), (190, 18): ([190, 170, 150, 151, 131, 132, 112, 113, 114, 94, 95, 75, 76, 77, 57, 37, 38, 18], 17), (342, 190): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 205, 206, 207, 208, 209, 210, 190], 16), (145, 190): ([145, 146, 166, 167, 168, 169, 170, 190], 7), (165, 190): ([165, 166, 167, 168, 169, 170, 190], 6), (122, 190): ([122, 123, 124, 125, 145, 146, 166, 167, 168, 169, 170, 190], 11), (132, 190): ([132, 131, 151, 150, 170, 190], 5), (127, 190): ([127, 128, 129, 149, 150, 170, 190], 6), (342, 127): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 165, 145, 146, 126, 127], 16), (145, 127): ([145, 146, 126, 127], 3), (165, 127): ([165, 145, 146, 126, 127], 4), (122, 127): ([122, 123, 124, 125, 105, 106, 107, 127], 7), (132, 127): ([132, 131, 151, 150, 149, 129, 128, 127], 7), (342, 132): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 165, 166, 167, 168, 148, 149, 150, 151, 131, 132], 21), (145, 132): ([145, 146, 126, 127, 128, 129, 149, 150, 151, 131, 132], 10), (165, 132): ([165, 166, 167, 168, 148, 149, 150, 151, 131, 132], 9), (122, 132): ([122, 123, 124, 125, 105, 106, 107, 108, 88, 89, 90, 110, 111, 131, 132], 14), (342, 122): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 144, 124, 123, 122], 15), (145, 122): ([145, 125, 124, 123, 122], 4), (165, 122): ([165, 145, 125, 124, 123, 122], 5), (342, 165): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 165], 12), (145, 165): ([145, 165], 1), (342, 145): ([342, 322, 323, 303, 283, 263, 243, 223, 224, 204, 184, 164, 165, 145], 13)} 
                                }},
                    {"inputs": {"maze": MazeFromDict({0: {20: 1}, 1: {2: 1}, 2: {1: 1, 3: 1, 22: 1}, 3: {2: 1, 4: 1, 23: 1}, 4: {3: 1, 5: 9, 24: 9}, 5: {6: 1, 25: 1, 4: 9}, 6: {7: 1, 26: 1, 5: 1}, 7: {6: 1, 8: 1, 27: 1}, 8: {9: 1, 7: 1, 28: 1}, 9: {8: 1, 29: 1, 10: 1}, 10: {30: 1, 11: 1, 9: 1}, 11: {10: 1, 31: 6}, 12: {13: 1, 32: 1}, 13: {12: 1, 14: 1, 33: 1}, 14: {13: 1}, 15: {16: 1, 35: 1}, 16: {15: 1, 17: 1, 36: 1}, 17: {18: 1, 16: 1}, 18: {17: 1, 19: 1, 38: 1}, 19: {18: 1, 39: 1}, 20: {0: 1, 40: 1, 21: 1}, 21: {22: 1, 20: 1, 41: 1}, 22: {21: 1, 42: 1, 2: 1, 23: 8}, 23: {43: 1, 24: 1, 3: 1, 22: 8}, 24: {25: 1, 23: 1, 44: 1, 4: 9}, 25: {24: 1, 26: 1, 45: 1, 5: 1}, 26: {6: 1, 25: 1, 46: 1, 27: 1}, 27: {47: 1, 7: 1, 26: 1}, 28: {29: 1, 8: 1}, 29: {9: 1, 28: 1, 30: 1, 49: 1}, 30: {10: 1, 29: 1, 50: 1}, 31: {51: 1, 11: 6}, 32: {33: 1, 12: 1, 52: 1}, 33: {13: 1, 32: 1, 53: 1, 34: 1}, 34: {54: 1, 35: 1, 33: 1}, 35: {15: 1, 36: 1, 55: 1, 34: 1}, 36: {35: 1, 37: 1, 16: 1}, 37: {57: 1, 36: 1, 38: 1}, 38: {18: 1, 39: 1, 58: 1, 37: 1}, 39: {38: 1, 19: 1, 59: 1}, 40: {20: 1, 60: 1, 41: 1}, 41: {42: 1, 61: 1, 21: 1, 40: 1}, 42: {41: 1, 43: 1, 22: 1, 62: 1}, 43: {23: 1, 42: 1, 63: 1, 44: 1}, 44: {64: 1, 45: 1, 24: 1, 43: 1}, 45: {25: 1, 44: 1, 65: 1, 46: 1}, 46: {26: 1, 66: 1, 45: 1}, 47: {27: 1, 67: 1, 48: 1}, 48: {49: 1, 68: 1, 47: 1}, 49: {29: 1, 48: 1, 69: 1, 50: 1}, 50: {70: 1, 30: 1, 49: 1, 51: 5}, 51: {31: 1, 52: 1, 71: 1, 50: 5}, 52: {51: 1, 53: 1, 32: 1}, 53: {33: 1, 54: 1, 52: 1, 73: 1}, 54: {34: 1, 53: 1, 74: 1, 55: 1}, 55: {35: 1, 56: 1, 54: 1, 75: 4}, 56: {55: 1, 57: 1, 76: 1}, 57: {37: 1, 56: 1, 77: 1, 58: 1}, 58: {38: 1, 59: 1, 78: 1, 57: 1}, 59: {58: 1, 79: 1, 39: 1}, 60: {40: 1, 80: 1, 61: 1}, 61: {62: 1, 41: 1, 60: 1, 81: 1}, 62: {61: 1, 82: 1, 42: 1}, 63: {43: 1, 64: 1, 83: 1}, 64: {44: 1, 63: 1, 65: 1, 84: 1}, 65: {64: 1, 66: 1, 45: 1}, 66: {46: 1, 65: 1, 67: 1, 86: 1}, 67: {47: 1, 66: 1, 68: 1, 87: 1}, 68: {67: 1, 69: 1, 88: 1, 48: 1}, 69: {49: 1, 68: 1, 70: 1, 89: 1}, 70: {69: 1, 71: 1, 90: 1, 50: 1}, 71: {51: 1, 70: 1, 91: 1}, 72: {92: 1}, 73: {93: 1, 53: 1}, 74: {54: 1, 75: 1, 94: 7}, 75: {95: 1, 74: 1, 55: 4}, 76: {56: 1, 96: 1, 77: 1}, 77: {78: 1, 97: 1, 76: 1, 57: 1}, 78: {58: 1, 77: 1, 98: 9}, 79: {59: 1, 99: 8}, 80: {60: 1, 81: 1, 100: 1}, 81: {80: 1, 82: 1, 101: 1, 61: 1}, 82: {62: 1, 81: 1, 102: 1, 83: 1}, 83: {103: 1, 82: 1, 63: 1}, 84: {104: 1, 85: 1, 64: 1}, 85: {105: 1, 84: 1, 86: 7}, 86: {87: 1, 66: 1, 85: 7}, 87: {67: 1, 107: 1, 86: 1, 88: 6}, 88: {68: 1, 108: 1, 89: 1, 87: 6}, 89: {109: 1, 90: 1, 69: 1, 88: 1}, 90: {91: 1, 89: 1, 70: 1, 110: 1}, 91: {71: 1, 90: 1, 111: 1, 92: 1}, 92: {72: 1, 112: 1, 91: 1, 93: 6}, 93: {73: 1, 113: 1, 94: 1, 92: 6}, 94: {114: 1, 93: 1, 74: 7}, 95: {75: 1, 96: 1, 115: 6}, 96: {76: 1, 97: 1, 95: 1, 116: 9}, 97: {77: 1, 117: 1, 96: 1, 98: 1}, 98: {118: 1, 99: 1, 97: 1, 78: 9}, 99: {98: 1, 119: 5, 79: 8}, 100: {101: 1, 120: 1, 80: 1}, 101: {100: 1, 121: 1, 81: 1}, 102: {82: 1, 103: 1, 122: 9}, 103: {83: 1, 102: 1, 104: 1, 123: 8}, 104: {84: 1, 103: 1}, 105: {85: 1, 125: 1, 106: 1}, 106: {126: 1, 105: 1, 107: 6}, 107: {127: 1, 108: 1, 87: 1, 106: 6}, 108: {128: 1, 107: 1, 109: 1, 88: 1}, 109: {89: 1, 110: 1, 108: 1, 129: 1}, 110: {109: 1, 111: 1, 130: 1, 90: 1}, 111: {91: 1, 110: 1, 131: 1, 112: 1}, 112: {92: 1, 132: 1, 111: 1, 113: 1}, 113: {93: 1, 114: 1, 133: 1, 112: 1}, 114: {94: 1, 113: 1, 115: 1, 134: 1}, 115: {114: 1, 116: 1, 135: 1, 95: 6}, 116: {115: 1, 136: 1, 117: 5, 96: 9}, 117: {97: 1, 118: 1, 137: 1, 116: 5}, 118: {98: 1, 117: 1, 138: 1}, 119: {139: 1, 99: 5}, 120: {100: 1, 140: 1}, 121: {101: 1, 122: 1, 141: 1}, 122: {121: 1, 123: 1, 142: 1, 102: 9}, 123: {122: 1, 143: 1, 124: 9, 103: 8}, 124: {125: 1, 144: 1, 123: 9}, 125: {105: 1, 124: 1, 126: 1}, 126: {125: 1, 146: 1, 106: 1, 127: 1}, 127: {107: 1, 147: 1, 128: 1, 126: 1}, 128: {108: 1, 129: 1, 127: 1, 148: 1}, 129: {128: 1, 130: 1, 149: 1, 109: 1}, 130: {129: 1, 131: 1, 110: 1, 150: 1}, 131: {111: 1, 130: 1, 151: 1, 132: 1}, 132: {133: 1, 112: 1, 152: 1, 131: 1}, 133: {132: 1, 134: 1, 113: 1, 153: 1}, 134: {114: 1, 133: 1, 154: 1, 135: 8}, 135: {136: 1, 115: 1, 155: 1, 134: 8}, 136: {116: 1, 135: 1, 137: 1, 156: 6}, 137: {117: 1, 138: 1, 157: 1, 136: 1}, 138: {137: 1, 118: 1, 158: 1, 139: 1}, 139: {119: 1, 159: 1, 138: 1}, 140: {160: 1, 120: 1, 141: 9}, 141: {121: 1, 142: 1, 161: 1, 140: 9}, 142: {141: 1, 143: 1, 122: 1, 162: 6}, 143: {163: 1, 142: 1, 123: 1, 144: 1}, 144: {124: 1, 145: 1, 143: 1, 164: 8}, 145: {146: 1, 144: 1, 165: 7}, 146: {126: 1, 145: 1, 147: 1, 166: 5}, 147: {127: 1, 167: 1, 148: 1, 146: 1}, 148: {168: 1, 147: 1, 128: 1, 149: 1}, 149: {150: 1, 148: 1, 169: 1, 129: 1}, 150: {149: 1, 170: 1, 151: 1, 130: 1}, 151: {131: 1, 171: 1, 150: 1, 152: 1}, 152: {172: 1, 132: 1, 151: 1, 153: 1}, 153: {154: 1, 173: 1, 133: 1, 152: 1}, 154: {134: 1, 153: 1, 155: 1, 174: 1}, 155: {154: 1, 175: 1, 135: 1}, 156: {176: 1, 157: 1, 136: 6}, 157: {158: 1, 156: 1, 137: 1, 177: 7}, 158: {157: 1, 138: 1, 178: 1}, 159: {139: 1, 179: 1}, 160: {140: 1, 161: 1, 180: 1}, 161: {141: 1, 160: 1, 162: 1, 181: 1}, 162: {161: 1, 163: 1, 182: 1, 142: 6}, 163: {143: 1, 162: 1, 164: 1, 183: 1}, 164: {163: 1, 184: 1, 165: 1, 144: 8}, 165: {185: 1, 164: 1, 145: 7}, 166: {167: 1, 186: 1, 146: 5}, 167: {147: 1, 166: 1, 187: 1, 168: 1}, 168: {148: 1, 169: 1, 167: 1, 188: 1}, 169: {168: 1, 189: 1, 170: 1, 149: 1}, 170: {150: 1, 190: 1, 169: 1}, 171: {151: 1, 172: 1, 191: 1}, 172: {152: 1, 171: 1, 192: 1, 173: 1}, 173: {174: 1, 193: 1, 153: 1, 172: 1}, 174: {154: 1, 173: 1, 175: 1, 194: 1}, 175: {174: 1, 155: 1, 176: 1, 195: 1}, 176: {156: 1, 175: 1, 177: 1, 196: 1}, 177: {197: 1, 176: 1, 157: 7}, 178: {179: 1, 158: 1}, 179: {159: 1, 178: 1, 199: 1}, 180: {160: 1, 181: 1}, 181: {201: 1, 161: 1, 182: 1, 180: 1}, 182: {162: 1, 202: 1, 183: 1, 181: 1}, 183: {203: 1, 182: 1, 163: 1, 184: 1}, 184: {204: 1, 164: 1, 183: 1}, 185: {165: 1, 186: 5}, 186: {187: 1, 166: 1, 185: 5, 206: 5}, 187: {167: 1, 188: 1, 186: 1, 207: 1}, 188: {187: 1, 189: 1, 208: 1, 168: 1}, 189: {169: 1, 188: 1, 190: 1, 209: 1}, 190: {170: 1, 189: 1, 191: 1, 210: 1}, 191: {171: 1, 190: 1, 211: 1, 192: 7}, 192: {193: 1, 212: 1, 172: 1, 191: 7}, 193: {192: 1, 173: 1, 213: 1, 194: 5}, 194: {174: 1, 195: 1, 214: 1, 193: 5}, 195: {194: 1, 196: 1, 175: 1}, 196: {197: 1, 195: 1, 216: 1, 176: 1}, 197: {177: 1, 196: 1, 217: 1, 198: 1}, 198: {199: 1, 197: 1, 218: 1}, 199: {179: 1, 198: 1, 219: 1}, 200: {201: 1, 220: 1}, 201: {181: 1, 200: 1, 202: 1, 221: 1}, 202: {182: 1, 201: 1, 203: 1, 222: 1}, 203: {183: 1, 202: 1, 223: 1, 204: 1}, 204: {184: 1, 224: 1, 203: 1, 205: 1}, 205: {206: 1, 204: 1, 225: 6}, 206: {205: 1, 207: 1, 226: 1, 186: 5}, 207: {206: 1, 227: 1, 187: 1, 208: 1}, 208: {207: 1, 188: 1, 228: 1, 209: 5}, 209: {189: 1, 229: 1, 210: 1, 208: 5}, 210: {190: 1, 230: 1, 209: 1, 211: 1}, 211: {212: 1, 191: 1, 210: 1, 231: 8}, 212: {192: 1, 211: 1, 213: 1, 232: 1}, 213: {212: 1, 233: 1, 193: 1}, 214: {194: 1, 234: 1, 215: 1}, 215: {216: 1, 235: 1, 214: 1}, 216: {215: 1, 217: 1, 196: 1, 236: 1}, 217: {197: 1, 218: 1, 237: 1, 216: 1}, 218: {217: 1, 219: 1, 198: 1}, 219: {199: 1, 218: 1, 239: 1}, 220: {200: 1, 221: 7}, 221: {201: 1, 222: 1, 241: 1, 220: 7}, 222: {202: 1, 223: 1, 221: 1, 242: 1}, 223: {203: 1, 224: 1, 222: 1, 243: 4}, 224: {204: 1, 223: 1, 225: 1, 244: 6}, 225: {224: 1, 245: 1, 226: 1, 205: 6}, 226: {227: 1, 206: 1, 246: 1, 225: 1}, 227: {207: 1, 226: 1, 228: 1, 247: 1}, 228: {227: 1, 248: 1, 208: 1, 229: 5}, 229: {230: 1, 209: 1, 228: 5}, 230: {210: 1, 229: 1, 231: 1, 250: 1}, 231: {230: 1, 232: 1, 251: 1, 211: 8}, 232: {212: 1, 231: 1, 252: 1, 233: 1}, 233: {234: 1, 213: 1, 253: 1, 232: 1}, 234: {214: 1, 233: 1, 235: 1, 254: 1}, 235: {215: 1, 234: 1, 236: 1, 255: 1}, 236: {235: 1, 237: 1, 256: 1, 216: 1}, 237: {217: 1, 236: 1, 238: 1, 257: 7}, 238: {237: 1, 239: 1, 258: 1}, 239: {238: 1, 219: 1, 259: 4}, 240: {241: 1, 260: 1}, 241: {240: 1, 242: 1, 261: 1, 221: 1}, 242: {241: 1, 243: 1, 222: 1, 262: 9}, 243: {242: 1, 263: 1, 244: 1, 223: 4}, 244: {264: 1, 243: 1, 245: 1, 224: 6}, 245: {225: 1, 265: 1, 246: 1, 244: 1}, 246: {247: 1, 245: 1, 226: 1, 266: 6}, 247: {227: 1, 246: 1, 248: 1, 267: 1}, 248: {247: 1, 249: 1, 228: 1, 268: 1}, 249: {248: 1, 250: 1, 269: 1}, 250: {249: 1, 251: 1, 270: 1, 230: 1}, 251: {231: 1, 250: 1, 271: 1, 252: 7}, 252: {253: 1, 272: 1, 232: 1, 251: 7}, 253: {252: 1, 254: 1, 273: 1, 233: 1}, 254: {234: 1, 253: 1, 255: 1, 274: 1}, 255: {254: 1, 256: 1, 275: 1, 235: 1}, 256: {255: 1, 236: 1, 276: 4}, 257: {258: 1, 237: 7, 277: 9}, 258: {278: 1, 257: 1, 238: 1, 259: 1}, 259: {279: 1, 258: 1, 239: 4}, 260: {240: 1, 261: 1, 280: 1}, 261: {241: 1, 262: 1, 260: 1, 281: 1}, 262: {261: 1, 263: 5, 242: 9}, 263: {243: 1, 283: 1, 262: 5, 264: 7}, 264: {244: 1, 265: 1, 284: 1, 263: 7}, 265: {245: 1, 264: 1, 285: 1, 266: 1}, 266: {286: 1, 267: 1, 265: 1, 246: 6}, 267: {247: 1, 268: 1, 287: 1, 266: 1}, 268: {267: 1, 288: 1, 248: 1}, 269: {270: 1, 289: 1, 249: 1}, 270: {269: 1, 271: 1, 250: 1}, 271: {251: 1, 270: 1, 291: 1, 272: 1}, 272: {252: 1, 273: 1, 271: 1}, 273: {253: 1, 274: 1, 293: 1, 272: 1}, 274: {273: 1, 275: 1, 294: 1, 254: 1}, 275: {274: 1, 255: 1, 276: 1, 295: 1}, 276: {296: 1, 275: 1, 256: 4, 277: 9}, 277: {278: 1, 297: 1, 257: 9, 276: 9}, 278: {258: 1, 277: 1, 279: 1, 298: 1}, 279: {278: 1, 259: 1, 299: 6}, 280: {300: 1, 260: 1, 281: 1}, 281: {282: 1, 280: 1, 261: 1, 301: 8}, 282: {281: 1, 302: 1}, 283: {284: 1, 263: 1, 303: 1}, 284: {264: 1, 283: 1, 285: 1, 304: 1}, 285: {265: 1, 286: 1, 305: 1, 284: 1}, 286: {285: 1, 287: 1, 306: 1, 266: 1}, 287: {267: 1, 286: 1, 307: 1}, 288: {308: 1, 268: 1, 289: 1}, 289: {269: 1, 309: 1, 288: 1}, 290: {291: 1, 310: 1}, 291: {271: 1, 290: 1, 292: 1, 311: 1}, 292: {291: 1, 312: 1, 293: 1}, 293: {273: 1, 313: 1, 292: 1}, 294: {274: 1, 295: 1, 314: 1}, 295: {294: 1, 315: 1, 275: 1, 296: 5}, 296: {276: 1, 316: 1, 295: 5}, 297: {277: 1, 298: 1}, 298: {297: 1, 318: 1, 278: 1}, 299: {319: 1, 279: 6}, 300: {280: 1, 301: 1, 320: 9}, 301: {300: 1, 302: 1, 321: 1, 281: 8}, 302: {282: 1, 301: 1, 322: 1, 303: 8}, 303: {323: 1, 283: 1, 302: 8}, 304: {305: 1, 324: 1, 284: 1}, 305: {285: 1, 304: 1, 325: 1}, 306: {286: 1, 307: 1, 326: 8}, 307: {287: 1, 327: 1, 308: 1, 306: 1}, 308: {288: 1, 309: 1, 328: 1, 307: 1}, 309: {289: 1, 308: 1, 329: 1, 310: 1}, 310: {290: 1, 330: 1, 311: 1, 309: 1}, 311: {291: 1, 331: 1, 310: 1}, 312: {292: 1, 332: 1}, 313: {314: 1, 333: 1, 293: 1}, 314: {313: 1, 315: 1, 334: 1, 294: 1}, 315: {314: 1, 316: 1, 335: 1, 295: 1}, 316: {315: 1, 317: 1, 296: 1, 336: 9}, 317: {316: 1, 337: 1, 318: 1}, 318: {298: 1, 319: 1, 317: 1, 338: 1}, 319: {299: 1, 339: 1, 318: 1}, 320: {340: 1, 321: 1, 300: 9}, 321: {301: 1, 341: 1, 322: 1, 320: 1}, 322: {323: 1, 302: 1, 321: 1, 342: 1}, 323: {303: 1, 322: 1, 324: 1, 343: 1}, 324: {323: 1, 325: 1, 344: 1, 304: 1}, 325: {305: 1, 324: 1, 345: 1, 326: 9}, 326: {346: 1, 327: 1, 325: 9, 306: 8}, 327: {307: 1, 347: 1, 326: 1, 328: 1}, 328: {308: 1, 348: 1, 329: 1, 327: 1}, 329: {309: 1, 349: 1, 328: 1, 330: 5}, 330: {310: 1, 350: 1, 331: 1, 329: 5}, 331: {311: 1, 351: 1, 330: 1}, 332: {312: 1, 352: 1, 333: 4}, 333: {313: 1, 353: 1, 334: 1, 332: 4}, 334: {314: 1, 354: 1, 333: 1}, 335: {315: 1, 336: 1}, 336: {335: 1, 337: 1, 316: 9, 356: 8}, 337: {317: 1, 357: 1, 338: 1, 336: 1}, 338: {339: 1, 337: 1, 358: 1, 318: 1}, 339: {319: 1, 338: 1, 359: 1}, 340: {320: 1, 341: 1, 360: 1}, 341: {321: 1, 340: 1, 361: 1, 342: 1}, 342: {343: 1, 341: 1, 322: 1, 362: 1}, 343: {323: 1, 342: 1, 344: 1, 363: 1}, 344: {324: 1, 345: 1, 343: 1, 364: 7}, 345: {344: 1, 346: 1, 365: 1, 325: 1}, 346: {326: 1, 366: 1, 345: 1, 347: 1}, 347: {327: 1, 367: 1, 348: 1, 346: 1}, 348: {328: 1, 368: 1, 349: 1, 347: 1}, 349: {329: 1, 369: 1, 350: 1, 348: 1}, 350: {330: 1, 370: 1, 349: 1, 351: 4}, 351: {331: 1, 352: 1, 350: 4}, 352: {351: 1, 353: 1, 372: 1, 332: 1}, 353: {352: 1, 373: 1, 333: 1, 354: 1}, 354: {334: 1, 355: 1, 353: 1}, 355: {354: 1, 375: 1, 356: 4}, 356: {357: 1, 376: 1, 355: 4, 336: 8}, 357: {337: 1, 356: 1, 377: 1, 358: 1}, 358: {359: 1, 357: 1, 378: 1, 338: 1}, 359: {339: 1, 358: 1, 379: 1}, 360: {340: 1, 380: 1}, 361: {341: 1, 362: 1, 381: 9}, 362: {363: 1, 361: 1, 342: 1}, 363: {362: 1, 364: 1, 343: 1, 383: 8}, 364: {363: 1, 365: 1, 384: 1, 344: 7}, 365: {364: 1, 385: 1, 345: 1, 366: 1}, 366: {346: 1, 386: 1, 367: 1, 365: 1}, 367: {347: 1, 366: 1, 387: 8}, 368: {388: 1, 348: 1, 369: 1}, 369: {349: 1, 370: 1, 389: 1, 368: 1}, 370: {369: 1, 371: 1, 390: 1, 350: 1}, 371: {370: 1, 372: 1, 391: 1}, 372: {352: 1, 392: 1, 371: 1}, 373: {353: 1, 374: 1, 393: 1}, 374: {373: 1, 375: 1, 394: 1}, 375: {374: 1, 376: 1, 395: 1, 355: 1}, 376: {375: 1, 356: 1, 377: 9}, 377: {357: 1, 397: 1, 376: 9}, 378: {398: 1, 358: 1, 379: 1}, 379: {359: 1, 399: 1, 378: 1}, 380: {381: 1, 360: 1}, 381: {380: 1, 361: 9}, 382: {383: 1}, 383: {382: 1, 384: 1, 363: 8}, 384: {364: 1, 383: 1, 385: 1}, 385: {365: 1, 386: 1, 384: 1}, 386: {366: 1, 387: 1, 385: 1}, 387: {386: 1, 388: 1, 367: 8}, 388: {368: 1, 387: 1, 389: 9}, 389: {369: 1, 390: 1, 388: 9}, 390: {370: 1, 389: 1, 391: 1}, 391: {392: 1, 371: 1, 390: 1}, 392: {372: 1, 391: 1, 393: 1}, 393: {392: 1, 394: 1, 373: 1}, 394: {393: 1, 374: 1, 395: 1}, 395: {375: 1, 396: 1, 394: 1}, 396: {395: 1}, 397: {377: 1, 398: 1}, 398: {378: 1, 397: 1, 399: 1}, 399: {398: 1, 379: 1}}),
                                "start": 110,
                                "peices_of_cheese":[300,80,97,114,360,250,224]},
                    "outputs": {"meta_graph":{(110, 300): ([110, 109, 108, 107, 127, 126, 125, 124, 144, 143, 142, 141, 161, 181, 201, 221, 241, 240, 260, 280, 300], 20), (110, 80): ([110, 90, 70, 69, 68, 67, 66, 65, 64, 63, 83, 82, 81, 80], 13), (110, 97): ([110, 111, 112, 113, 114, 115, 116, 136, 137, 117, 97], 10), (110, 114): ([110, 111, 112, 113, 114], 4), (110, 360): ([110, 109, 108, 107, 127, 147, 167, 187, 207, 206, 226, 225, 245, 244, 243, 263, 283, 303, 323, 322, 321, 320, 340, 360], 23), (110, 250): ([110, 130, 150, 170, 190, 210, 230, 250], 7), (110, 224): ([110, 109, 108, 107, 127, 147, 167, 187, 207, 206, 205, 204, 224], 12), (224, 300): ([224, 223, 222, 221, 241, 240, 260, 280, 300], 8), (224, 80): ([224, 204, 184, 164, 163, 143, 123, 122, 121, 101, 81, 80], 11), (224, 97): ([224, 204, 205, 206, 207, 187, 167, 147, 148, 149, 150, 151, 152, 153, 154, 155, 135, 136, 137, 117, 97], 20), (224, 114): ([224, 204, 205, 206, 207, 187, 167, 147, 127, 107, 108, 109, 110, 111, 112, 113, 114], 16), (224, 360): ([224, 223, 222, 221, 241, 240, 260, 280, 300, 301, 321, 320, 340, 360], 13), (224, 250): ([224, 225, 226, 227, 228, 248, 249, 250], 7), (250, 300): ([250, 249, 248, 247, 246, 245, 244, 243, 242, 241, 240, 260, 280, 300], 13), (250, 80): ([250, 230, 210, 190, 170, 150, 149, 148, 147, 146, 145, 144, 143, 123, 122, 121, 101, 81, 80], 18), (250, 97): ([250, 230, 210, 190, 170, 150, 151, 152, 153, 154, 155, 135, 136, 137, 117, 97], 15), (250, 114): ([250, 230, 210, 190, 170, 150, 130, 110, 111, 112, 113, 114], 11), (250, 360): ([250, 249, 248, 247, 246, 245, 244, 243, 263, 283, 303, 323, 322, 321, 320, 340, 360], 16), (360, 300): ([360, 340, 320, 321, 301, 300], 5), (360, 80): ([360, 340, 320, 321, 301, 300, 280, 260, 240, 241, 221, 201, 181, 161, 141, 121, 101, 81, 80], 18), (360, 97): ([360, 340, 320, 321, 322, 323, 303, 283, 263, 243, 244, 245, 225, 226, 206, 207, 187, 167, 147, 148, 149, 150, 151, 152, 153, 154, 155, 135, 136, 137, 117, 97], 31), (360, 114): ([360, 340, 320, 321, 322, 323, 303, 283, 263, 243, 244, 245, 225, 226, 206, 207, 187, 167, 147, 127, 107, 108, 109, 110, 111, 112, 113, 114], 27), (114, 300): ([114, 113, 112, 111, 110, 109, 108, 107, 127, 126, 125, 124, 144, 143, 142, 141, 161, 181, 201, 221, 241, 240, 260, 280, 300], 24), (114, 80): ([114, 113, 112, 92, 91, 71, 70, 69, 68, 67, 66, 65, 64, 63, 83, 82, 81, 80], 17), (114, 97): ([114, 115, 116, 136, 137, 117, 97], 6), (97, 300): ([97, 117, 137, 136, 135, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 161, 181, 201, 221, 241, 240, 260, 280, 300], 28), (97, 80): ([97, 77, 57, 56, 55, 54, 53, 52, 51, 71, 70, 69, 68, 67, 66, 65, 64, 63, 83, 82, 81, 80], 21), (80, 300): ([80, 81, 101, 121, 141, 161, 181, 201, 221, 241, 240, 260, 280, 300], 13), (300, 110): ([300, 280, 260, 240, 241, 221, 201, 181, 161, 141, 142, 143, 144, 124, 125, 126, 127, 107, 108, 109, 110], 20), (80, 110): ([80, 81, 82, 83, 63, 64, 65, 66, 67, 68, 69, 70, 90, 110], 13), (97, 110): ([97, 117, 137, 136, 116, 115, 114, 113, 112, 111, 110], 10), (114, 110): ([114, 113, 112, 111, 110], 4), (360, 110): ([360, 340, 320, 321, 322, 323, 303, 283, 263, 243, 244, 245, 225, 226, 206, 207, 187, 167, 147, 127, 107, 108, 109, 110], 23), (250, 110): ([250, 230, 210, 190, 170, 150, 130, 110], 7), (224, 110): ([224, 204, 205, 206, 207, 187, 167, 147, 127, 107, 108, 109, 110], 12), (300, 224): ([300, 280, 260, 240, 241, 221, 222, 223, 224], 8), (80, 224): ([80, 81, 101, 121, 122, 123, 143, 163, 164, 184, 204, 224], 11), (97, 224): ([97, 117, 137, 136, 135, 155, 154, 153, 152, 151, 150, 149, 148, 147, 167, 187, 207, 206, 205, 204, 224], 20), (114, 224): ([114, 113, 112, 111, 110, 109, 108, 107, 127, 147, 167, 187, 207, 206, 205, 204, 224], 16), (360, 224): ([360, 340, 320, 321, 301, 300, 280, 260, 240, 241, 221, 222, 223, 224], 13), (250, 224): ([250, 249, 248, 228, 227, 226, 225, 224], 7), (300, 250): ([300, 280, 260, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250], 13), (80, 250): ([80, 81, 101, 121, 122, 123, 143, 144, 145, 146, 147, 148, 149, 150, 170, 190, 210, 230, 250], 18), (97, 250): ([97, 117, 137, 136, 135, 155, 154, 153, 152, 151, 150, 170, 190, 210, 230, 250], 15), (114, 250): ([114, 113, 112, 111, 110, 130, 150, 170, 190, 210, 230, 250], 11), (360, 250): ([360, 340, 320, 321, 322, 323, 303, 283, 263, 243, 244, 245, 246, 247, 248, 249, 250], 16), (300, 360): ([300, 301, 321, 320, 340, 360], 5), (80, 360): ([80, 81, 101, 121, 141, 161, 181, 201, 221, 241, 240, 260, 280, 300, 301, 321, 320, 340, 360], 18), (97, 360): ([97, 117, 137, 136, 135, 155, 154, 153, 152, 151, 150, 149, 148, 147, 167, 187, 207, 206, 226, 225, 245, 244, 243, 263, 283, 303, 323, 322, 321, 320, 340, 360], 31), (114, 360): ([114, 113, 112, 111, 110, 109, 108, 107, 127, 147, 167, 187, 207, 206, 226, 225, 245, 244, 243, 263, 283, 303, 323, 322, 321, 320, 340, 360], 27), (300, 114): ([300, 280, 260, 240, 241, 221, 201, 181, 161, 141, 142, 143, 144, 124, 125, 126, 127, 107, 108, 109, 110, 111, 112, 113, 114], 24), (80, 114): ([80, 81, 82, 83, 63, 64, 65, 66, 67, 68, 69, 70, 71, 91, 92, 112, 113, 114], 17), (97, 114): ([97, 117, 137, 136, 116, 115, 114], 6), (300, 97): ([300, 280, 260, 240, 241, 221, 201, 181, 161, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 135, 136, 137, 117, 97], 28), (80, 97): ([80, 81, 82, 83, 63, 64, 65, 66, 67, 68, 69, 70, 71, 51, 52, 53, 54, 55, 56, 57, 77, 97], 21), (300, 80): ([300, 280, 260, 240, 241, 221, 201, 181, 161, 141, 121, 101, 81, 80], 13)}
                                }}]
                    
            # Test on several controlled examples
            for i in range(len(TESTS)):
                
                # Instantiate the player
                player =Two_opt()

                # Get the input and output data
                maze = TESTS[i]["inputs"]["maze"]
                start = TESTS[i]["inputs"]["start"]
                peices_of_cheese=TESTS[i]["inputs"]["peices_of_cheese"]
                target_meta_graph = TESTS[i]["outputs"]["meta_graph"]

                # Perform the traversal
                meta_graph = player.meta_graph(maze,peices_of_cheese,start)

                # Check that outputs match the expected ones
                self.assertEqual(sorted(meta_graph), sorted(target_meta_graph))

    def test_two_opt(self:Self):
        # Constants
        INPUTS = [{"meta_graph":{8: {14: (2, [8, 9, 14]), 1: (11, [8, 7, 6, 1]), 12: (2, [8, 7, 12]), 17: (3, [8, 7, 12, 17]), 0: (12, [8, 7, 6, 1, 0])}, 0: {14: (14, [0, 1, 6, 7, 8, 9, 14]), 1: (1, [0, 1]), 12: (12, [0, 1, 6, 7, 12]), 17: (13, [0, 1, 6, 7, 12, 17]), 8: (12, [0, 1, 6, 7, 8])}, 17: {14: (5, [17, 12, 7, 8, 9, 14]), 1: (12, [17, 12, 7, 6, 1]), 12: (1, [17, 12]), 8: (3, [17, 12, 7, 8]), 0: (13, [17, 12, 7, 6, 1, 0])}, 12: {14: (4, [12, 7, 8, 9, 14]), 1: (11, [12, 7, 6, 1]), 8: (2, [12, 7, 8]), 0: (12, [12, 7, 6, 1, 0]), 17: (1, [12, 17])}, 1: {14: (13, [1, 6, 7, 8, 9, 14]), 8: (11, [1, 6, 7, 8]), 0: (1, [1, 0]), 17: (12, [1, 6, 7, 12, 17]), 12: (11, [1, 6, 7, 12])}, 14: {8: (2, [14, 9, 8]), 0: (14, [14, 9, 8, 7, 6, 1, 0]), 17: (5, [14, 9, 8, 7, 12, 17]), 12: (4, [14, 9, 8, 7, 12]), 1: (13, [14, 9, 8, 7, 6, 1])}} ,
                       "partial_path":[14, 8, 12, 17, 1, 0]
                       },
                  {"meta_graph":{16: {5: (3, [16, 15, 10, 5]), 14: (9, [16, 17, 18, 19, 14]), 4: (11, [16, 17, 18, 19, 14, 9, 4]), 17: (1, [16, 17]), 13: (12, [16, 17, 18, 13])}, 13: {5: (15, [13, 18, 17, 16, 15, 10, 5]), 14: (6, [13, 14]), 4: (8, [13, 14, 9, 4]), 17: (11, [13, 18, 17]), 16: (12, [13, 18, 17, 16])}, 17: {5: (4, [17, 16, 15, 10, 5]), 14: (8, [17, 18, 19, 14]), 4: (10, [17, 18, 19, 14, 9, 4]), 16: (1, [17, 16]), 13: (11, [17, 18, 13])}, 4: {5: (14, [4, 9, 14, 19, 18, 17, 16, 15, 10, 5]), 14: (2, [4, 9, 14]), 16: (11, [4, 9, 14, 19, 18, 17, 16]), 13: (8, [4, 9, 14, 13]), 17: (10, [4, 9, 14, 19, 18, 17])}, 14: {5: (12, [14, 19, 18, 17, 16, 15, 10, 5]), 16: (9, [14, 19, 18, 17, 16]), 13: (6, [14, 13]), 17: (8, [14, 19, 18, 17]), 4: (2, [14, 9, 4])}, 5: {16: (3, [5, 10, 15, 16]), 13: (15, [5, 10, 15, 16, 17, 18, 13]), 17: (4, [5, 10, 15, 16, 17]), 4: (14, [5, 10, 15, 16, 17, 18, 19, 14, 9, 4]), 14: (12, [5, 10, 15, 16, 17, 18, 19, 14])}},
                       "partial_path":[5, 16, 17, 14, 4, 13]
                       }]
        OUTPUTS=[[14, 8, 12, 17, 1, 0],[5, 16, 17, 14, 4, 13]]

        # Test on several controlled examples
        for i in range(len(INPUTS)):
            
            # Instantiate the player
            player = Two_opt()
            output=OUTPUTS[i]

            # Get the input data
            meta_graph = INPUTS[i]["meta_graph"]
            partial_path= INPUTS[i]["partial_path"]
            

            # Find the route
            route = player.complete_path(partial_path,meta_graph)

            # Check the output type
            # It should be a list of integers
            self.assertTrue(isinstance(route, List))
            self.assertTrue(all(isinstance(v, Integral) for v in route))

            # Check that the route is a valid path from the start to the end
            self.assertTrue(route,output)         


<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [7]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_traversal (__main__.Two_optTests.test_traversal)
This method tests the 'traversal' method. ... ERROR
test_traversal_fixed (__main__.Two_optTests.test_traversal_fixed)
This method complements the previous tests by testing the 'traversal' method on some controlled examples. ... ERROR
test_two_opt (__main__.Two_optTests.test_two_opt) ... ok

ERROR: test_traversal (__main__.Two_optTests.test_traversal)
This method tests the 'traversal' method.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\pp\AppData\Local\Temp\ipykernel_1760\3772481177.py", line 60, in test_traversal
    distances, routing_table = player.traversal(maze, start_vertex)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Two_opt.traversal() missing 1 required positional argument: 'targets'

ERROR: test_traversal_fixed (__main__.Two_optTests.test_traversal_fixed)
This method complements the previous tests by testing the